In [10]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from plyfile import PlyData, PlyElement
import tqdm
import meshplot as mp
import mesh_tools as mt
try: 
    import sys 
    sys.path.append('../')
    from radfoam import Triangulation
    from radfoam_model.render import TraceRays
    from radfoam_model.utils import *
    print('USING RADFOAM CUDA KERNEL')
except:
    from scipy.spatial import Delaunay
    class Triangulation(nn.Module):
        def __init__(self, points):
            super().__init__()
            self.points = points
            self.update_scipy_delaunay()
            self.repulsion_fac = 100
            
        def permutation(self):
            return torch.arange(len(self.points), device=self.points.device).long()
        
        def tets(self):
            return torch.tensor(self.delaunay.simplices, dtype=torch.float32, device=self.points.device)
            
        def tets_to_edges(self):
            tets = self.tets()
            edges = []
            edges.append(torch.column_stack((tets[:, 0], tets[:, 1])))
            edges.append(torch.column_stack((tets[:, 0], tets[:, 2])))
            edges.append(torch.column_stack((tets[:, 0], tets[:, 3])))
            edges.append(torch.column_stack((tets[:, 1], tets[:, 3])))
            edges.append(torch.column_stack((tets[:, 2], tets[:, 3])))
            edges.append(torch.column_stack((tets[:, 1], tets[:, 2])))
            return torch.cat(edges).view(-1,6,2)
            
        def update_scipy_delaunay(self):
            self.delaunay = Delaunay(self.points.cpu().detach().numpy())
          

USING RADFOAM CUDA KERNEL


In [11]:
num_init_points = 100000
primal_points = (
    torch.randn(num_init_points, 3, device='cuda') * 25
)


In [12]:
triangulation = Triangulation(primal_points)
perm = triangulation.permutation().to(torch.long)
primal_points = primal_points[perm]

In [13]:
primal_values = (torch.norm(primal_points,dim=1) < 30) * 1.0 - 0.5
mp.plot(*mt.marching_tetraheadra(triangulation.tets().long(),primal_values,primal_points))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.042716…